# Mengzi-T5-base Finetune 


In [2]:
!pip install datasets bitsandbytes evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 4.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 32.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 56.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 77.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 26.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 kB 16.

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, EarlyStoppingCallback
from evaluate import load
import numpy as np
import torch

In [ ]:
csv_path = "train.csv"
df = pd.read_csv(csv_path)

examples = []
for _, row in df.iterrows():
    if pd.notna(row.get("Pinyin")) and pd.notna(row.get("Chinese")):
        examples.append({
            "input": row["Pinyin"],
            "target": row["Chinese"]
        })
dataset = Dataset.from_pandas(pd.DataFrame(examples))

train_size = int(0.8 * len(dataset))
eval_size = len(dataset) - train_size
train_subset, eval_subset = torch.utils.data.random_split(dataset, [train_size, eval_size])

train_dataset = Dataset.from_list([dataset[i] for i in train_subset.indices])
eval_dataset = Dataset.from_list([dataset[i] for i in eval_subset.indices])
del dataset

In [4]:
model_name = "Langboat/mengzi-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/725k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    prefix = "拼音转中文："
    inputs = [prefix + text for text in examples["input"]]
    targets = examples["target"] # Chinese

    model_inputs = tokenizer(
        inputs,
        padding="longest",
        truncation=True,
    )

    labels = tokenizer(
        text_target=targets,
        truncation=True,
        padding="longest"
    )

    label_ids = [
        [-100 if token == tokenizer.pad_token_id else token for token in label]
        for label in labels["input_ids"]
    ]
    model_inputs["labels"] = label_ids

    return model_inputs

# Tokenize the entire dataset
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


Map:   0%|          | 0/14959 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/3740 [00:00<?, ? examples/s]

In [ ]:
# Optional BLEU for comparison
bleu = load("bleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    print("Pred shape:", np.array(preds).shape)
    print("Labels shape:", np.array(labels).shape)

    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)

    # clip predictions to valid token ID range
    preds = np.clip(preds, 0, tokenizer.vocab_size - 1)

    invalid = [(i, val) for i, row in enumerate(labels) for val in row if val < 0 or val >= tokenizer.vocab_size]
    if invalid:
        print("Invalid token IDs found:", invalid[:5])  # print just a few for now
        raise ValueError("Found token ids out of tokenizer vocab range.")

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Normalize whitespace, remove special tokens, etc. if needed
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # Character-level accuracy
    char_correct = 0
    char_total = 0
    for pred, label in zip(decoded_preds, decoded_labels):
        char_total += len(label)
        char_correct += sum(p == l for p, l in zip(pred, label))

    char_accuracy = char_correct / char_total if char_total > 0 else 0.0

    # BLEU (optional, use for logging or reference)
    bleu_result = bleu.compute(predictions=decoded_preds, references=[[lbl] for lbl in decoded_labels])
    bleu_score = bleu_result["bleu"]

    return {
        "char_accuracy": char_accuracy,
        "bleu": bleu_score
    }

training_args = Seq2SeqTrainingArguments(
    output_dir="./t5_pinyin_to_chinese", 
    evaluation_strategy="steps",
    learning_rate=3e-5,
    per_device_train_batch_size=16,       
    per_device_eval_batch_size=16,     
    weight_decay=0.01,
    save_total_limit=2,             
    num_train_epochs=20,          
    predict_with_generate=True,
    load_best_model_at_end=True,
    greater_is_better=True,
    metric_for_best_model="char_accuracy", 
    gradient_accumulation_steps=8,
    save_steps=200,                        
    logging_steps=100,                     
    fp16=False,
    bf16=True,
    eval_steps=200,                        
    save_strategy="steps",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback()]
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-b2caf9a513ef>:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()
trainer.evaluate()
# # ----------------- VALIDATE -------------------
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_path = "./t5_pinyin_to_chinese_final"  # Adjust path if different
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
model.to(device)

test_csv_path = "eval.csv"
df_test = pd.read_csv(test_csv_path)

if "Pinyin" not in df_test.columns:
    raise ValueError("The CSV file must contain a column with Chinese words.")

def generate_chinese_text(pinyin_sentence):
    input_text = "拼音转中文：" + pinyin_sentence
    input_ids = tokenizer(input_text, return_tensors="pt", padding="longest", truncation=True)["input_ids"]
    input_ids = torch.LongTensor(input_ids).view(1, -1).to(model.device)

    generated_ids = model.generate(input_ids, max_new_tokens=64)
    prediction = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    print(f"Pinyin: {pinyin_sentence}\nChinese: {prediction}\n")

    return prediction

df_test["Predicted Chinese"] = df_test["Pinyin"].apply(generate_chinese_text)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pipiroy03 (pipiroy03-simon-fraser-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

Pred shape: (3740, 21)
Labels shape: (3740, 40)
Pred shape: (3740, 21)
Labels shape: (3740, 40)
Pred shape: (3740, 21)
Labels shape: (3740, 40)
Pred shape: (3740, 21)
Labels shape: (3740, 40)
Pred shape: (3740, 21)
Labels shape: (3740, 40)
Pred shape: (3740, 21)
Labels shape: (3740, 40)
Pred shape: (3740, 21)
Labels shape: (3740, 40)
Pred shape: (3740, 21)
Labels shape: (3740, 40)
Pred shape: (3740, 21)
Labels shape: (3740, 40)
Pred shape: (3740, 21)
Labels shape: (3740, 40)
Pred shape: (3740, 21)
Labels shape: (3740, 40)


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


Pred shape: (3740, 21)
Labels shape: (3740, 40)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Pinyin: ta xi huan na ta de tong xue kai wan xiao
Chinese: 他喜欢拿他的奖学金开玩笑

Pinyin: ru guo you ji hui wo jiang qu kan na bu dian ying
Chinese: 如果有机会我将去看那部电影

Pinyin: cong zhe ge jiao du lai kan hua zhong de nv zi mian dai wei xiao
Chinese: 从这个教堂来观赏中的女主人公微小

Pinyin: ta song wo de ji nian pin hen xiao er qie bu zhi qian
Chinese: 她送我的几件旧品很小而且毫不在乎

Pinyin: zhe ben shu shi ta yi qian xie de ju ben de kuo chong
Chinese: 这本书是他以前写的集本的缩微

Pinyin: wo men zhen de chu yu kun jing mei ren zhao gu ying er
Chinese: 我们真的处于困境没人照顾婴儿

Pinyin: ta shi huo de yi xue jiao ke shu
Chinese: 他是获得医学博士学位

Pinyin: wo dong yi dian er de yu
Chinese: 我动一下耳朵的语

Pinyin: ci li shi yi zhong zi ran xian xiang
Chinese: 此处是一种自然现象

Pinyin: qi shi nian dai shi chao duan qun shi dai
Chinese: 去年年代是极端人群

Pinyin: ji qi sheng chan yi jing dai ti le shou gong lao zuo
Chinese: 机器生产已经代替了手工业工作

Pinyin: lao zi shuang fang zhi jian cun zai da liang mao dun
Chinese: 老房子建在大半个村子

Pinyin: kong pa wo de fang wei gan hen cha yin ci wo rong yi mi 

In [ ]:
trainer.save_model("./mengzi_t5_base_pinyin_to_chinese_final")

# Misc

In [ ]:
del dataset

In [ ]:
!zip -r mengzi_t5_base_pinyin_to_chinese_final.zip mengzi_t5_base_pinyin_to_chinese_final//

  adding: t5_pinyin_to_chinese_final// (stored 0%)
  adding: t5_pinyin_to_chinese_final//config.json (deflated 48%)
  adding: t5_pinyin_to_chinese_final//generation_config.json (deflated 29%)
  adding: t5_pinyin_to_chinese_final//added_tokens.json (deflated 83%)
  adding: t5_pinyin_to_chinese_final//model.safetensors (deflated 7%)
  adding: t5_pinyin_to_chinese_final//training_args.bin (deflated 51%)
  adding: t5_pinyin_to_chinese_final//tokenizer_config.json (deflated 94%)
  adding: t5_pinyin_to_chinese_final//spiece.model (deflated 46%)
  adding: t5_pinyin_to_chinese_final//special_tokens_map.json (deflated 85%)


In [ ]:
from google.colab import files
files.download("/content/mengzi_t5_pinyin_to_chinese.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>